In [1]:
import os
from hdfs.ext.kerberos import KerberosClient
import getpass
import uuid
import time

In [2]:
_NAMENODE_PORT = 50070

In [3]:
def get_client(namenode_url: str, namenode_port: int = None):
    port = namenode_port = _NAMENODE_PORT
    return KerberosClient('%s:%s' % (namenode_url, port))


In [4]:
def upload(local_file: str,
           namenode_url: str,
           namenode_port: int = None,
           hdfs_dir: str = None,):
    """
    Parameters
    ----------
    local_file: The local file to upload
    namenode_url: The url of the namenode
    namenode_port: (optional) The port that the namenode service is running on. Defaults to 
        value of module-level variable _NAMENODE_PORT.
    hdfs_dir: (optional) The folder (or full path) to upload `local_file` to. If none is provided,
        then the file will be uploaded to the following directory: 
        /tmp/livy_{current user name}_{First six characters of uuid4}_{current time in seconds}'
        
    Returns
    -------
    str: The directory where `local_file` was uploaded so you can use it to upload the rest of your
        files needed for your Spark job
    """
    port = namenode_port or _NAMENODE_PORT
    
    client = get_client(namenode_url, port)
    
    if hdfs_dir is None:
        hdfs_dir = f'/tmp/livy_{getpass.getuser()}_{str(uuid.uuid4())[:6]}_{round(time.time())}/'
        
#     print(hdfs_dir)
    client.makedirs(hdfs_dir)
    
    resp = client.upload(hdfs_dir, local_file, overwrite=True)
#     print(resp)
    
    return hdfs_dir
    
    
    


---

Do some QA on the hdfs client functions

In [ ]:
namenode_url = 

In [114]:
namenode_url = 'http://ip-172-31-20-241.ec2.internal'

In [115]:
upload('../QA/pi.py', namenode_url)

http://ip-172-31-20-241.ec2.internal:50070
/tmp/livy_ec2-user_dfe804_1545062025/
/tmp/livy_ec2-user_dfe804_1545062025/pi.py


'/tmp/livy_ec2-user_dfe804_1545062025/'

In [125]:
client.delete('/tmp/livy_ec2-user_dfe804_1545062025/', recursive=True)

True

In [116]:
client = get_client(namenode_url)

http://ip-172-31-20-241.ec2.internal:50070


In [118]:
client.list('/tmp/livy_ec2-user_dfe804_1545062025/')

['pi.py']

In [57]:
client._get_home_directory??

Signature: client._get_home_directory(hdfs_path, data=None, strict=True, **params)
Docstring: Cf. http://hadoop.apache.org/docs/r1.0.4/webhdfs.html#GETHOMEDIRECTORY
Source:   
    def api_handler(client, hdfs_path, data=None, strict=True, **params):
      """Wrapper function."""
      params['op'] = operation
      if client._proxy is not None:
        params['doas'] = client._proxy

      attempted_hosts = set()
      while True:
        with client._lock:
          while client._urls[0] in attempted_hosts:
            client._urls.rotate(-1)
          host = client._urls[0]
        url = '%s%s%s' % (
          host.rstrip('/'),
          self.webhdfs_prefix,
          quote(client.resolve(hdfs_path), '/= '),
        )

        err = None
        try:
          res = client._request(
            method=self.method,
            url=url,
            data=data,
            params=params,
            **self.kwargs
          )
        except (rq.exceptions.ReadTimeout, rq.exceptions.Connec

In [26]:
client.upload?a

Signature:
client.upload(
    ['hdfs_path', 'local_path', 'n_threads=1', 'temp_dir=None', 'chunk_size=65536', 'progress=None', 'cleanup=True', '**kwargs'],
)
Docstring:
Upload a file or directory to HDFS.

:param hdfs_path: Target HDFS path. If it already exists and is a
  directory, files will be uploaded inside.
:param local_path: Local path to file or folder. If a folder, all the files
  inside of it will be uploaded (note that this implies that folders empty
  of files will not be created remotely).
:param n_threads: Number of threads to use for parallelization. A value of
  `0` (or negative) uses as many threads as there are files.
:param temp_dir: Directory under which the files will first be uploaded
  when `overwrite=True` and the final remote path already exists. Once the
  upload successfully completes, it will be swapped in.
:param chunk_size: Interval in bytes by which the files will be uploaded.
:param progress: Callback function to track progress, called every
  `chunk_si

In [130]:
import sys
sys.path.insert(0, '../')

In [131]:
from livy_submit import hdfs_api

In [132]:
namenode_url = 'http://ip-172-31-20-241.ec2.internal:50070'

In [133]:
local_file = '../test/data/pi_runner.zip'

In [142]:
hdfs_dirname = '/user/edill/livy-submit-files/19d579_1545492416'

In [143]:
hdfs_api.upload(local_file, namenode_url, hdfs_dirname)

file uploaded to /user/edill/livy-submit-files/19d579_1545492416/pi_runner.zip


'/user/edill/livy-submit-files/19d579_1545492416/pi_runner.zip'